Đây là hai phiên bản cải tiến của Linear Regression, được thiết kế để giải quyết overfitting và xử lý dữ liệu có nhiều đặc trưng — đặc biệt khi các đặc trưng tương quan cao (đa cộng tuyến) hoặc số đặc trưng > số mẫu.

Mình sẽ trình bày theo đúng phong cách quen thuộc:

🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Vấn đề với Linear Regression thông thường:
Khi có nhiều đặc trưng, hoặc đặc trưng tương quan mạnh, Linear Regression có thể:
Overfit (học thuộc nhiễu)
Hệ số rất lớn → mô hình không ổn định
🧠 Giải pháp: Thêm "hình phạt" vào hàm mất mát
Cả Ridge và Lasso đều thêm một thành phần phạt vào lỗi MSE, để kìm hãm độ lớn của hệ số.

Ridge Regression
L2 Regularization
Phạt
tổng bình phương hệ số
:
w 
1
2
​
 +w 
2
2
​
 +…
Lasso Regression
L1 Regularization
Phạt
tổng giá trị tuyệt đối hệ số
:
∣w 
1
​
 ∣+∣w 
2
​
 ∣+…

💡 Hiệu ứng khác biệt:
Ridge:
Giảm hệ số về gần 0, nhưng không bao giờ = 0.
Giúp ổn định mô hình, giảm overfit.
Lasso:
Ép một số hệ số = 0 → tự động loại bỏ đặc trưng không quan trọng.
Vừa chống overfit, vừa chọn đặc trưng (feature selection).
🌟 Không cần nhớ công thức!
Chỉ cần nhớ: 

Ridge = làm nhỏ hệ số
Lasso = làm nhỏ + loại bỏ đặc trưng
✅ Ưu điểm chung:
Chống overfit hiệu quả.
Ổn định hơn Linear Regression khi dữ liệu có nhiễu.
Hoạt động tốt khi số đặc trưng lớn.
❌ Hạn chế:
Cần chọn tham số phạt (alpha) → dùng cross-validation.
Mất tính chất "giải thích đơn giản" của Linear Regression (vì hệ số bị co).
🔹 3. Khi nào dùng Ridge vs Lasso?
Tất cả đặc trưng đều có ích
, chỉ muốn
giảm overfit
✅
Ridge
Nhiều đặc trưng không quan trọng
, muốn
tự động chọn đặc trưng
✅
Lasso
Số đặc trưng > số mẫu
✅
Lasso
(Ridge cũng hoạt động, nhưng không chọn đặc trưng)
Đặc trưng tương quan mạnh
✅
Ridge
(Lasso có thể chọn ngẫu nhiên 1 trong số chúng)
Muốn kết hợp cả hai
→ Dùng
ElasticNet
(kết hợp L1 + L2)
🎯 So sánh nhanh với Linear Regression:
Chống overfit
❌ Không
✅ Có
✅ Có
Chọn đặc trưng
❌ Không
❌ Không
✅ Có
Hệ số = 0
Hiếm
Không
✅ Có
Ổn định với đặc trưng tương quan
Kém
✅ Tốt
Trung bình
Cần chuẩn hóa
Nên
✅
Bắt buộc
✅
Bắt buộc


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

class RidgeRegressor:
    def __init__(self, alpha=1.0, learning_rate=0.01, n_iters=1000):
        self.alpha = alpha  # hệ số phạt
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_pred = np.dot(X, self.weights) + self.bias
            # Gradient của MSE + L2 penalty
            dw = (1/n_samples) * np.dot(X.T, (y_pred - y)) + (2 * self.alpha * self.weights)
            db = (1/n_samples) * np.sum(y_pred - y)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias


class LassoRegressor:
    def __init__(self, alpha=1.0, learning_rate=0.01, n_iters=1000):
        self.alpha = alpha
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def _soft_threshold(self, w, alpha, lr):
        """Soft thresholding cho L1 penalty"""
        if w > lr * alpha:
            return w - lr * alpha
        elif w < -lr * alpha:
            return w + lr * alpha
        else:
            return 0.0

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_pred = np.dot(X, self.weights) + self.bias
            # Cập nhật bias (không phạt)
            db = (1/n_samples) * np.sum(y_pred - y)
            self.bias -= self.lr * db

            # Cập nhật weights với soft thresholding
            for j in range(n_features):
                dw = (1/n_samples) * np.dot(X[:, j], (y_pred - y))
                w_new = self.weights[j] - self.lr * dw
                self.weights[j] = self._soft_threshold(w_new, self.alpha, self.lr)

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# --- Thử nghiệm ---
# Tạo dữ liệu có nhiều đặc trưng, một số không quan trọng
X, y = make_regression(n_samples=100, n_features=20, noise=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Chuẩn hóa (quan trọng với regularization)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Huấn luyện các mô hình
lr = RidgeRegressor(alpha=1.0, learning_rate=0.1, n_iters=2000)
lr.fit(X_train, y_train)
y_pred_ridge = lr.predict(X_test)

lasso = LassoRegressor(alpha=0.5, learning_rate=0.1, n_iters=2000)
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

# So sánh với sklearn
from sklearn.linear_model import Ridge, Lasso
sk_ridge = Ridge(alpha=1.0)
sk_ridge.fit(X_train, y_train)
sk_lasso = Lasso(alpha=0.5, max_iter=2000)
sk_lasso.fit(X_train, y_train)

# Đánh giá
print("✅ Ridge (tự code) R²:", r2_score(y_test, y_pred_ridge))
print("✅ Lasso (tự code) R²:", r2_score(y_test, y_pred_lasso))
print("🔍 Sklearn Ridge R²:", sk_ridge.score(X_test, y_test))
print("🔍 Sklearn Lasso R²:", sk_lasso.score(X_test, y_test))

# Xem hệ số Lasso (nhiều hệ số = 0!)
print("\n🌟 Số hệ số = 0 trong Lasso (tự code):", np.sum(np.abs(lasso.weights) < 1e-4))
print("🌟 Số hệ số = 0 trong Sklearn Lasso:", np.sum(sk_lasso.coef_ == 0))

✅ Ridge (tự code) R²: 0.5809065137456728
✅ Lasso (tự code) R²: 0.9896262658008363
🔍 Sklearn Ridge R²: 0.9893860041183064
🔍 Sklearn Lasso R²: 0.989627001327533

🌟 Số hệ số = 0 trong Lasso (tự code): 4
🌟 Số hệ số = 0 trong Sklearn Lasso: 4
